In [ ]:
import csv
import re
from google.colab import drive
import os
import pandas as pd
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import nltk
import ast
import json

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
inverted_index = {}

In [ ]:
ALT_WEIGHT = 2
RELATED_WEIGHT = 1

In [ ]:
def get_weight(strength):
    if strength > 0.9:
        return 3
    elif strength > 0.6:
        return 2
    return 1

In [ ]:
def read_json_file(file_path):
    """
    Read a JSON file and return its contents as a Python object.

    Args:
    file_path (str): The path to the JSON file.

    Returns:
    dict: The contents of the JSON file as a Python dictionary.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [ ]:
tagged_directory = "/content/drive/MyDrive/MOS_2/Tagged"
# tagged_directory = "/content/drive/MyDrive/MOS_2/Tagged_v2"
tagged_directory = "/content/drive/MyDrive/MOS_2/Tagged_v3"
files = os.listdir(tagged_directory)

In [ ]:
tagged_csv_files = [tagged_directory + "/" + file for file in files if file.endswith('.csv')]
print(len(tagged_csv_files))

229


In [ ]:
# # Initialize lemmatizer and stopwords
# lemmatizer = WordNetLemmatizer()
# stop_words = set(stopwords.words('english'))

# # Function to tokenize text
# def tokenize_text(text):
#     tokens = word_tokenize(text)
#     tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalnum() and token.lower() not in stop_words]
#     return tokens

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function to tokenize text
def tokenize_text(text):
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(token.lower()) for token in tokens if token.isalnum() and token.lower() not in stop_words]
    return tokens


print(tokenize_text("a man running in the mountains"))

['man', 'run', 'mountain']


In [ ]:
def convert_to_valid_json(text):
    # Convert text to Python object
    obj = ast.literal_eval(text)
    return obj

In [ ]:
# df = pd.read_csv(tagged_csv_files[0])
df = pd.read_csv("/content/drive/MyDrive/MOS_2/Tagged/mountain landscape_image_data.csv")
df = df.dropna()
df.head()

,Alt,URL,Title,Anchor_Href,Relative_Title,Relative_URL,Related_Images,image_tags,Unique_ID
0,body of water and snow-covered mountains durin...,https://images.unsplash.com/photo-148049749078...,body of water and snow-covered mountains durin...,/photos/body-of-water-and-snow-covered-mountai...,body of water and snow-covered mountains durin...,/photos/body-of-water-and-snow-covered-mountai...,"[{'Text': 'Hd wallpapers', 'Relative_Href': '/...","[{'alp': 0.6950416}, {'valley': 0.15378875}, {...",63426d9a-0650-45b1-a6d2-8c3f9ea9918a
2,gray concrete road between trees near mountain,https://images.unsplash.com/photo-147805929987...,gray concrete road between trees near mountain,/photos/gray-concrete-road-between-trees-near-...,gray concrete road between trees near mountain,/photos/gray-concrete-road-between-trees-near-...,"[{'Text': 'road', 'Relative_Href': '/s/photos/...","[{'alp': 0.6639409}, {'valley': 0.05882883}, {...",2bbf9986-0694-4414-9924-5ed672e4e434
3,white mountain near body of water,https://images.unsplash.com/photo-150178622340...,white mountain near body of water,/photos/white-mountain-near-body-of-water-Q5dM...,white mountain near body of water,/photos/white-mountain-near-body-of-water-Q5dM...,"[{'Text': 'Landscape images & pictures', 'Rela...","[{'alp': 0.38585272}, {'valley': 0.2207979}, {...",bc3e3115-d580-4ecd-8496-914c167dbfbd
4,water stream surrounded with green trees,https://images.unsplash.com/photo-149885592648...,water stream surrounded with green trees,/photos/water-stream-surrounded-with-green-tre...,water stream surrounded with green trees,/photos/water-stream-surrounded-with-green-tre...,"[{'Text': 'Nature images', 'Relative_Href': '/...","[{'valley': 0.95239764}, {'lakeside': 0.011717...",7cba454f-7fbb-45b4-bf2d-e236f926376a
5,snow-covered tree near body of water,https://images.unsplash.com/photo-145515621838...,snow-covered tree near body of water,/photos/snow-covered-tree-near-body-of-water-r...,snow-covered tree near body of water,/photos/snow-covered-tree-near-body-of-water-r...,"[{'Text': 'Nature images', 'Relative_Href': '/...","[{'valley': 0.48332578}, {'alp': 0.46678263}, ...",68482297-4e89-4a0e-aacb-046d44d72f27


In [ ]:
df.drop(columns=['Title', 'Relative_Title', 'Relative_URL', 'Anchor_Href', 'URL'], inplace=True)
df['Alt'] = df['Alt'].apply(tokenize_text)

In [ ]:
df.head()

,Alt,Related_Images,image_tags,Unique_ID
0,body of water and snow-covered mountains durin...,"[{'Text': 'Hd wallpapers', 'Relative_Href': '/...","[{'alp': 0.6950416}, {'valley': 0.15378875}, {...",63426d9a-0650-45b1-a6d2-8c3f9ea9918a
1,NaN,"[{'Text': 'Nature images', 'Relative_Href': '/...",NaN,b129f11d-5ce4-4fd9-b4a4-351c8a94b60d
2,gray concrete road between trees near mountain,"[{'Text': 'road', 'Relative_Href': '/s/photos/...","[{'alp': 0.6639409}, {'valley': 0.05882883}, {...",2bbf9986-0694-4414-9924-5ed672e4e434
3,white mountain near body of water,"[{'Text': 'Landscape images & pictures', 'Rela...","[{'alp': 0.38585272}, {'valley': 0.2207979}, {...",bc3e3115-d580-4ecd-8496-914c167dbfbd
4,water stream surrounded with green trees,"[{'Text': 'Nature images', 'Relative_Href': '/...","[{'valley': 0.95239764}, {'lakeside': 0.011717...",7cba454f-7fbb-45b4-bf2d-e236f926376a


In [ ]:
def update_inverted_index(record):
    local_tokens = []
    local_tokens.extend(record['Alt'] * ALT_WEIGHT)

    # related_images_str = record['Related_Images'].replace("'", "\"")

    related_images_list = convert_to_valid_json(record['Related_Images'])
    # Adding related images
    for related_image in related_images_list:
        # local_tokens.extend(tokenize_text(related_image['Text']) * RELATED_WEIGHT)
        local_tokens.extend(tokenize_text(related_image) * RELATED_WEIGHT)

    # image_tags_list = json.loads(record['image_tags'].replace("'", "\""))
    image_tags_list = convert_to_valid_json(record['image_tags'])

    # Adding tags with varying strengths
    for tag_info in image_tags_list:
        for tag, strength in tag_info.items():
            local_tokens.extend(tokenize_text(tag) * get_weight(strength))

    # Count frequency of each token
    token_frequency = {}
    for token in local_tokens:
        token_frequency[token] = token_frequency.get(token, 0) + 1

    # Update inverted index
    for token, frequency in token_frequency.items():
        if token not in inverted_index:
            inverted_index[token] = {}
        if record['Unique_ID'] in inverted_index[token]:
            inverted_index[token][record['Unique_ID']] += frequency
        else:
            inverted_index[token][record['Unique_ID']] = frequency


    return

In [ ]:
print( dict(sorted(inverted_index['brown'].items(), key=lambda item: item[1], reverse=True)))

In [ ]:
def generate_index():
    for csv_file in tagged_csv_files:
      print("generating index for: ", csv_file)
      df = pd.read_csv(csv_file)
      df = df.dropna()
      df = df.drop_duplicates(subset=['URL'])
      df.drop(columns=['Title', 'Relative_Title', 'Relative_URL', 'Anchor_Href', 'URL'], inplace=True)
      df['Alt'] = df['Alt'].apply(tokenize_text)

      for index in df.index:
        row = df.loc[index]
        update_inverted_index(row)
      print("Done.\n")

In [ ]:
# inverted_index = read_json_file("/content/drive/MyDrive/MOS_2/Index/inverted_index.json")
inverted_index = read_json_file("/content/drive/MyDrive/MOS_2/Index/inverted_index_v2.json")

In [ ]:
print(inverted_index['black'])

{'90d31fab-feaf-4056-b23a-5efb3df637ad': 2, 'd53485ab-c6c0-49b6-97f7-1a66a2222e7f': 2, '849ae88f-1692-422e-baaa-558cbe87782c': 3, '89a99fde-87df-490d-b814-2cea5533aa33': 3, '37cd0fbf-9641-4df4-87a0-8c7e3806fc54': 2, '951a453b-7006-4559-ab75-05f79a414afb': 2, '5ba1495e-b906-456e-b268-b76566da0900': 1, '44c3992a-de86-4a70-a29d-5b21c9cc51ec': 1, '6d293622-a554-47f3-8822-e05c858dfdcb': 2, 'c90c3de5-b659-480d-91a6-fab461d950e3': 2, 'c63be184-434b-4045-acb6-b6f26f683640': 2, 'a80cdd41-af06-45bd-893b-82bd92a0f55c': 3, '45b20db7-9f3d-4ba2-b7e0-0180523fe424': 1, '9a1830e4-4985-4632-a11b-debf5ffc31b5': 2, '77347f14-c1f9-4463-80cc-2ffb1e08c77e': 3, '013ecdc0-55fb-4f8f-991c-c888d69ad8a9': 2, '5150408f-14ef-402c-805b-cc9ac70ea4a3': 2, '33a8196c-aec6-46c1-91b9-75061fb82bae': 1, '63350476-c100-4a8c-a035-9f795d75efaf': 1, '9b9e5151-e36a-4a61-82c0-cfce36bdc87d': 2, '421fa004-78b7-4f8c-adaa-f64104033412': 1, 'eec5557c-6856-4370-b1cb-38f673b4312c': 2, '56bb65b6-b154-4bfc-9ec8-bc4bbc25be0e': 3, 'ec58aceb-

In [ ]:
generate_index()

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/paper_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/students_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/beautiful hands_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/pen_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/africans_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/hands_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/scenery_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/petals_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/humans_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/code_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/google_image_data.csv

<ipython-input-12-ccb0815c6b7e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Title', 'Relative_Title', 'Relative_URL', 'Anchor_Href', 'URL'], inplace=True)
<ipython-input-12-ccb0815c6b7e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Alt'] = df['Alt'].apply(tokenize_text)


Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/film genres_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/art galleries_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/movie reviews_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/art exhibitions_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/cinematography_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/art criticism_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/film festivals_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/music festivals_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/movie soundtracks_image_data.csv
Done.

generating index for:  /content/drive/MyDrive/MOS_2/Tagged_v3/concerts_image_data.csv
Done.

generating

In [ ]:
def save_json_file(file_path):
    with open(file_path, 'w') as json_file:
        json.dump(inverted_index, json_file, indent=4)
    return "Done."

save_json_file("/content/drive/MyDrive/MOS_2/Index/inverted_index_v3.json")

'Done.'